In [8]:
import os

In [9]:
folder_path='/home/jsy/shengyao_dataset/bags'
dirs=os.listdir('/home/jsy/shengyao_dataset/bags')
dir_paths=[]
for dir in dirs:
    dir_path=os.path.join(folder_path,dir)
    dir_paths.append(dir_path)
# file_paths=[]
# for foldername, subfolders, filenames in os.walk(folder_path):
#     for filename in filenames:
#         file_path = os.path.join(foldername, filename)
#         file_paths.append(file_path)

In [10]:
import re

In [11]:
pattern='(.*?)_2024'
re.search(pattern, dirs[0], flags=0).group(1)

'1_Town10HD_0.2_200_SoftRainNoon'

In [12]:
dirs

['1_Town10HD_0.2_200_SoftRainNoon_2024-09-08-21-04',
 '2_Town10HD_0.2_200_SoftRainNoon_2024-09-08-21-06',
 '3_Town10HD_0.2_200_SoftRainNoon_2024-09-08-21-08']

In [13]:
from rosbags.rosbag2 import Reader
from rosbags.typesys import Stores, get_typestore
import os
import cv2
from PIL import Image
from rosbags.image import message_to_cvimage
import numpy as np
import random

In [14]:
folder_count=0
for dir in dirs:
    image_count=0
    output_dir = '/home/jsy/val2_datasets'
    source_dir='/home/jsy/shengyao_dataset/bags'
    os.makedirs(output_dir, exist_ok=True)
    pattern='(.*?)_2024'
    folder_name=re.search(pattern, dir, flags=0).group(1)
    output_dir_train_x=output_dir+f'/{folder_name}'+'/train_x'
    output_dir_train_y=output_dir+f'/{folder_name}'+'/train_y'
    output_dir_test_x=output_dir+f'/{folder_name}'+'/test_x'
    output_dir_test_y=output_dir+f'/{folder_name}'+'/test_y'
    os.makedirs(output_dir_train_x, exist_ok=True)
    os.makedirs(output_dir_train_y, exist_ok=True)
    os.makedirs(output_dir_test_x, exist_ok=True)
    os.makedirs(output_dir_test_y, exist_ok=True)
    # Create a typestore.
    typestore = get_typestore(Stores.LATEST)
    
    
    # Create reader instance and open for reading.
    with Reader(os.path.join(source_dir,dir)) as reader:
        # Topic and msgtype information is available on .connections list.
        train_x = []
        train_y = []
        time1=0
        time2=1
        time1_prev=0
        time2_prev=1
        for connection in reader.connections:
            print(connection.topic, connection.msgtype)
        # Iterate over messages.
        for connection, timestamp, rawdata in reader.messages():
            if connection.topic == '/carla/ego_vehicle/rgb/image':
                msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
                # get opencv image and convert to bgr8 color space
                img = message_to_cvimage(msg, "bgr8")
                im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                time1 = msg.header.stamp.nanosec
            else:
                pass
    
            if connection.topic == '/carla/ego_vehicle/segmentation/image':
                msg_2 = typestore.deserialize_cdr(rawdata, connection.msgtype)
                # get opencv image and convert to bgr8 color space
                img_2 = message_to_cvimage(msg_2, "bgr8")
                im_rgb_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
                time2 = msg_2.header.stamp.nanosec
            else:
                 pass
    
            if time1==time2:
                if time1 != time1_prev and time2 != time2_prev:
                    if random.random()<=0.7:
                        Image.fromarray(im_rgb).save(f'{output_dir_train_x}/image_{image_count:04d}.png')
                        Image.fromarray(im_rgb_2).save(f'{output_dir_train_y}/image_{image_count:04d}.png')
                        image_count+=1
                        time1_prev = time1 
                        time2_prev = time2
                        #print(time1)
                    else:
                        Image.fromarray(im_rgb).save(f'{output_dir_test_x}/image_{image_count:04d}.png')
                        Image.fromarray(im_rgb_2).save(f'{output_dir_test_y}/image_{image_count:04d}.png')
                        image_count+=1
                        time1_prev = time1 
                        time2_prev = time2
                        #print(time1) 
                else:
                    pass
            else:
                pass
        folder_count+=1
        print('finished %d datasets' % folder_count)

/carla/ego_vehicle/rgb/image sensor_msgs/msg/Image
/carla/ego_vehicle/segmentation/image sensor_msgs/msg/Image
/tf tf2_msgs/msg/TFMessage
finished 1 datasets
/carla/ego_vehicle/rgb/image sensor_msgs/msg/Image
/carla/ego_vehicle/segmentation/image sensor_msgs/msg/Image
/tf tf2_msgs/msg/TFMessage
finished 2 datasets
/carla/ego_vehicle/rgb/image sensor_msgs/msg/Image
/carla/ego_vehicle/segmentation/image sensor_msgs/msg/Image
/tf tf2_msgs/msg/TFMessage
finished 3 datasets
